In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability.python.bijectors import affine_scalar
import math as m
import random as r
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import h5py
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from keras import backend as K

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
K.set_floatx('float64')

In [ ]:
# sigh https://github.com/keras-team/keras/issues/4875
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)



In [ ]:
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors
tfk = tf.keras

In [ ]:
SIG_NAMES = {
    '1': {
        1: "stop2b1000_neutralino300",
        2: "glgl1400_neutralino1100",
        28: "monojet_Zp2000.0_DM_50.0",
        29: "glgl1600_neutralino800",
        30: "monotop_200_A",
        31: "stlp_st1000",
        32: "sqsq_sq1800_neut800",
        33: "sqsq1_sq1400_neut800",
        999: "Secret"
    },
    '2a': {
        25: "chaneut_cha250_neut150",
        26: "chaneut_cha400_neut200",
        27: "pp24mt_50",
        28: "pp23mt_50",
        29: "gluino_1000.0_neutralino_1.0",
        30: "chaneut_cha200_neut50",
        31: "chaneut_cha300_neut100",
        999: "Secret"
    },
    '2b': {
        1: "pp24mt_50",
        2: "chaneut_cha200_neut50",
        3: "stlp_st1000",
        4: "chacha_cha600_neut200",
        5: "pp23mt_50",
        6: "chaneut_cha250_neut150",
        7: "chacha_cha400_neut60",
        34: "gluino_1000.0_neutralino_1.0",
        35: "chacha_cha300_neut140",
        999: "Secret"
    },
    '3': {
        1: "glgl1600_neutralino800",
        2: "monojet_Zp2000.0_DM_50.0",
        3: "gluino_1000.0_neutralino_1.0",
        4: "stop2b1000_neutralino300",
        5: "sqsq1_sq1400_neut800",
        6: "monotop_200_A",
        7: "monoV_Zp2000.0_DM_1.0",
        8: "stlp_st1000",
        34: "sqsq_sq1800_neut800",
        35: "glgl1400_neutralino1100",
        999: "Secret"
    }
}

In [ ]:
SIG_TYPES = {
    '1': [1, 2, 28, 29, 30, 31, 32, 33,999],
    '2a': [25, 26, 27, 28, 29, 30, 31,999],
    '2b': [1,2,3,4,5,6,7, 34,35,999],
    '3': [1,2,3,4,5,6,7,8,34,35,999]
}

"""
class types:
0 none
1 j
2 b
3 g
4 e+
5 e-
6 m+
7 m-
"""

CLASS = {
    'none': 0.,
    'j': 1.,
    'b': 2.,
    'g': 3.,
    'e+': 4.,
    'e-': 5.,
    'm+': 6.,
    'm-': 7.
}
MAX_JET = 7
MAX_LEP = 4

CHANNELS = ['1', '2a', '2b', '3']
n_epochs = 100

def update_points(x_reg, x_class, energies):
    
    MET = x_reg[0]
    METphi = x_reg[1]
    num_j = sum(x_class == CLASS['j'])
    num_b = sum(x_class == CLASS['b'])
    num_g = sum(x_class == CLASS['g'])
    num_ep = sum(x_class == CLASS['e+'])
    num_em = sum(x_class == CLASS['e-'])
    num_mp = sum(x_class == CLASS['m+'])
    num_mm = sum(x_class == CLASS['m-'])
    
    num_j = np.random.uniform(num_j-0.5, num_j+0.5)
    num_b = np.random.uniform(num_b-0.5, num_b+0.5)
    num_g = np.random.uniform(num_g-0.5, num_g+0.5)
    num_ep = np.random.uniform(num_ep-0.5, num_ep+0.5)
    num_em = np.random.uniform(num_em-0.5, num_em+0.5)
    num_mp = np.random.uniform(num_mp-0.5, num_mp+0.5)
    num_mm = np.random.uniform(num_mm-0.5, num_mm+0.5)
    
    # Get jets
    jets_ar = np.random.uniform(-6, -3, MAX_JET * 5)
    jets_indices = np.where((x_class == CLASS['j']) | (x_class == CLASS['b']))[0]
        
    it=0
    while it < np.min((len(jets_indices), MAX_JET)):
        i = jets_indices[it]
        reg_vals = x_reg[i*4:i*4+4]
        reg_type = x_class[i]
        jets_ar[it*5] = np.random.uniform(-1, 0) if reg_type == CLASS['b'] else np.random.uniform(0, 1)
        jets_ar[it*5+1:it*5+5] = reg_vals
        it+=1
        
    # Get leptons
    lep_ar = np.random.uniform(-6, -3, MAX_LEP * 5)
    lep_indices = np.where((x_class == CLASS['e+']) | (x_class == CLASS['e-']) | (x_class == CLASS['g']) | (x_class == CLASS['m+']) | (x_class == CLASS['m-']))[0]
        
    it=0
    while it < np.min((len(lep_indices), MAX_LEP)):
        i = lep_indices[it]
        reg_vals = x_reg[i*4:i*4+4]
        reg_type = x_class[i]
        
        if reg_type == CLASS['m-']:
            lep_ar[it*5] = np.random.uniform(-2.5, -1.5)
        if reg_type == CLASS['e-']:
            lep_ar[it*5] = np.random.uniform(-1.5, -0.5)
        if reg_type == CLASS['g']:
            lep_ar[it*5] = np.random.uniform(-0.5, 0.5)
        if reg_type == CLASS['e+']:
            lep_ar[it*5] = np.random.uniform(0.5, 1.5)
        if reg_type == CLASS['m+']:
            lep_ar[it*5] = np.random.uniform(1.5, 2.5)
                         
        lep_ar[it*5+1:it*5+5] = reg_vals
        it+=1
    
    row = [MET, METphi, num_j, num_b, num_g, num_ep, num_em, num_mp, num_mm]
#     row.extend(list(jets_ar))
#     row.extend(list(lep_ar))
    return row

def get_data(channel):
    train_h5 = h5py.File('training_chan'+str(channel)+'.h5', 'r')
    train_raw_reg = np.array(train_h5['reg'], dtype=np.float64)
    train_raw_class = np.array(train_h5['clas'], dtype=np.float64)
    energies = list(i*4+2 for i in range(int((train_raw_reg.shape[1]-1)/4)))
    
    train = []
    for i in tqdm(range(len(train_raw_reg))):
        train.append(update_points(train_raw_reg[i], train_raw_class[i,:,0], energies))
    train = np.array(train)
    
    print(train.shape,energies)
    
    test_h5 = h5py.File('testing_chan'+str(channel)+'.h5', 'r')
    secret_h5 = h5py.File('inference_chan'+str(channel)+'.h5', 'r')
    
    test_bg_reg = []
    
    test_sig_reg = []
    signal_types = []

    
    types = test_h5['type']
    reg = test_h5['reg']
    clas = test_h5['clas']
    
    for i in tqdm(range(len(types))):
        typ = types[i]
        reg_updated = update_points(reg[i], clas[i,:,0], energies)
        if typ in SIG_TYPES[channel]:
            # signal
            test_sig_reg.append(reg_updated)
            signal_types.append(typ)
        else:
            # bg
            test_bg_reg.append(reg_updated)
            
    # Append secret set
    secret_clas = secret_h5['clas']
    secret_reg = secret_h5['reg']
    for i in tqdm(range(len(secret_reg))):
        signal_types.append(999)
        reg_updated = update_points(secret_reg[i], secret_clas[i,:,0], energies)
        test_sig_reg.append(reg_updated)
    
    test_bg = np.array(test_bg_reg, dtype=np.float64)
    test_sig = np.array(test_sig_reg, dtype=np.float64)
        
    return train, test_bg, test_sig, signal_types


train, test_bg, test_sig, signal_types = get_data('2a')
print(train[0])
print('done')

In [ ]:
def batched_log_prob(dist, inputs, batch_size=1000):
    num_batches = m.ceil(inputs.shape[0] / batch_size)
    result = []
    for b in range(num_batches):
        result.extend(dist.log_prob(inputs[b*batch_size:(b+1)*batch_size]))
    result = np.array(result)
    return result
    
CHANNELS=['3']
for channel_num in CHANNELS:
    background_train_unscaled, background_test_unscaled, signal_unscaled, signal_types = get_data(channel_num)
    
    # Scale to unit box
    epsilon = 1e-4 # Small factor to ensure no training points are exactly on the edge of phase space
    max = np.amax( np.concatenate((background_train_unscaled, background_test_unscaled, signal_unscaled)), axis=0 )*(1+epsilon) + epsilon 
    min = np.amin( np.concatenate((background_train_unscaled, background_test_unscaled, signal_unscaled)), axis=0 )*(1+epsilon) - epsilon
    background_train = (background_train_unscaled - min)/(max - min)
    background_test = (background_test_unscaled - min)/(max - min)
    signal_flat = (signal_unscaled - min)/(max - min)

    event_dim = background_train.shape[1]

    n_RQS_knots = 35 # Number of knots in RQS transform
    n_made_layers = 7 # Number of layers in every made network
    n_made_units = 200 # Number of units in every layer of the made network
    n_flow_layers = 11 # Number of layers in the flow
    # Training parameters
    batch_size = 512

    def create_bijector_fn(made=None):
        # If no MADE is provided, create a default one
        hidden_units_ = [n_made_units]*n_made_layers
        if made is None:
            made = tfb.AutoregressiveNetwork(params=3*n_RQS_knots-1,
                                             input_order='random',
                                             activation=tf.nn.leaky_relu,
                                             hidden_units=hidden_units_,
                                             dtype=tf.float64)
        # Define the custom_bijector_fn
        # This function will use the provided (or just created) MADE internally due to scope
        def custom_bijector_fn(x):
            bw_pre, bh_pre, ks_pre = tf.split(made(x), [n_RQS_knots, n_RQS_knots, n_RQS_knots-1], axis=-1)
            bw = tf.math.softmax(bw_pre, axis=-1) * (1 - n_RQS_knots*1e-3) + 1e-3
            bh = tf.math.softmax(bh_pre, axis=-1) * (1 - n_RQS_knots*1e-3) + 1e-3
            ks = tf.math.softplus(ks_pre) + 1e-3
            return tfb.RationalQuadraticSpline(bin_widths=bw, bin_heights=bh, knot_slopes=ks, range_min=0.)
        # Return handle to the just defined function
        return custom_bijector_fn

    # Base distribution
    dist = tfd.Uniform(low=np.float64(0.), high=np.float64(1.))

    # Loop for number of flow layers
    for i in range(n_flow_layers):
        event_shape_ = [event_dim] if i==0 else None
        # Replace the distribution with an additional layer
        dist = tfd.TransformedDistribution(
            distribution = dist, 
            bijector = tfb.MaskedAutoregressiveFlow(bijector_fn = create_bijector_fn()),
            event_shape = event_shape_)

    # Construct model
    x_ = tfk.layers.Input(shape=(event_dim,), dtype=tf.float64)
    log_prob = dist.log_prob(x_)
    model = tfk.Model(x_, log_prob)

    model.compile(optimizer=tf.optimizers.Adam(epsilon=1e-3),
                  loss=lambda _, log_prob: tf.clip_by_value(-log_prob, -1e9, 100))

    epoch_num = n_epochs
    if channel_num == '3':
        epoch_num = 10
    
    losses = model.fit(x = background_train, y = np.zeros((background_train.shape[0], 0), dtype=np.float32), batch_size=batch_size, epochs=epoch_num, shuffle=True, verbose=True)
    np.savetxt('effenc_small_loss_' + channel_num, losses.history['loss'])

#     Compute log probs for signal and background

    
    logprobs_background = batched_log_prob(dist, background_test)
    logprobs_background[logprobs_background == -np.inf] = np.amin(logprobs_background[logprobs_background != -np.inf]) + np.sign(logprobs_background[logprobs_background == -np.inf])*1e-4 # Kick out spurious -inf 
    
    # For every signal separately
    signals = {}
    for typ in np.unique(signal_types):
        signals[typ] = []
    for i in range(len(signal_flat)):
        signals[signal_types[i]].append(signal_flat[i])
        
    model.save_weights('model_weights_effenc_small_flow_' + channel_num)
#     model.load_weights('model_weights_effenc_flow_' + channel_num)
        
    fpr_list = {}
    tpr_list = {}
    logprobs_signal_list = {}
    roc_auc_list = {}
    for signal_type in signals:
        signal = np.array(signals[signal_type])
        sig_name = SIG_NAMES[channel_num][signal_type]
    
        logprobs_signal = batched_log_prob(dist, signal)
        logprobs_signal[logprobs_signal == -np.inf] = np.amin(logprobs_signal[logprobs_signal != -np.inf]) + np.sign(logprobs_signal[logprobs_signal == -np.inf])*1e-4 # Kick out spurious -inf 

        # Normalize logprobs to a score
        max_log_prob = np.amax(np.concatenate((logprobs_background, logprobs_signal)))
        min_log_prob = np.amin(np.concatenate((logprobs_background, logprobs_signal)))
        background_scores = (logprobs_background - min_log_prob)/(max_log_prob - min_log_prob)
        signal_scores = (logprobs_signal - min_log_prob)/(max_log_prob - min_log_prob)
        
        print(background_test_unscaled[0])
        print(signal_unscaled[0])
        
        np.savetxt('scores_effenc_final_small_'+str(channel_num)+'_bg.csv', background_scores)
        np.savetxt('scores_effenc_final_small_'+str(channel_num)+'_'+str(signal_type)+'.csv', signal_scores)


        # Concatenate to total scores and labels
        total_scores = np.concatenate((background_scores, signal_scores))
        background_labels = np.ones_like(background_scores)
        signal_labels = np.zeros_like(signal_scores)
        total_labels = np.concatenate((background_labels, signal_labels))

        # Compute roc curve
        fpr, tpr, thresholds = roc_curve(total_labels, total_scores)
        roc_auc = roc_auc_score(y_true=total_labels, y_score=total_scores)
        print("Area under ROC curve for " + str(sig_name) + " is ", roc_auc)
        
        fpr_list[signal_type] = fpr
        tpr_list[signal_type] = tpr
        logprobs_signal_list[signal_type] = logprobs_signal
        roc_auc_list[signal_type] = roc_auc

    

    fig = plt.figure(figsize=(10,10))
    for key in fpr_list:
        sig_name = SIG_NAMES[channel_num][key]
        plt.plot(fpr_list[key], tpr_list[key], label='ROC curve '+ str(sig_name) +' (area = %0.3f)' % roc_auc_list[key])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    fig.savefig("effenc_small_ROC"+channel_num+".png", bbox_inches='tight', dpi=150)

    fig = plt.figure(figsize=(10,10))
    plt.xlabel('Log_Likelihood')
    for key in logprobs_signal_list:
        sig_name = SIG_NAMES[channel_num][key]
        plt.hist(logprobs_signal_list[key], bins=25, alpha=0.5, density=True, label="signal " + str(sig_name))
    plt.hist(logprobs_background, bins=25, alpha=0.5, density=True, label="background")
    plt.legend(loc='lower left')
    fig.savefig("effenc_small_likelihood_histogram"+channel_num+".png", bbox_inches='tight', dpi=150)
